<div style="text-align: center;">
<h1>The University of North Carolina at Chapel Hill</h1>
<h1>Comp 555 BioAlgorithms - Spring 2021</h1>
<h1 style="font-size: 250%;">Problem Set #3</h1>
<h1 style="font-size: 150%;">Version 1.1</h1>
<h1>Issued Thursday, 2/22/2021; Due Thursday, 3/16/2021</h1>
</div>

**Homework Information:** Some of the problems are probably too long to be done the night before the due date, so plan accordingly. Late problem sets will not be accepted. Feel free to get help from others, but **the work you submit in should be your own.**

**Warning:** This notebook has been annotated with metadata so that it can be uploaded to the grading system. It is very important that you enter your answers in the provided cells. You can add extra cells to explore approaches, but only the provided cell can and will be graded. Thus, if you delete a cell and add a replacement, there is a possiblity that your problem will not be graded. If you ever need to start over, you should download a new version of the problem set and transfer your solutions to it.

In [2]:
# Replace the following string values with the requested information
class Student:
    first = "Katelyn"
    last = "Cline"
    onyen = "katelyn9"
    pid = "730152239"

You will need a set of 31 base-pair <a href="http://csbio.unc.edu/mcmillan/Comp555S21/reads.fa" download="reads.fa">simulated reads</a> from a genome that will be used for assembly.
The cell below provides various functions and imports necessary for this problem set. Do not import any addtional packages. Also, make sure that you ***run the following cell***.

In many problems you are asked to consider or report an interval of a substring. I am expecting you to use Python's interval/range definition as is used for slicing strings and accessing lists. Such interval specifications are called *right-open* meaning that they contain all values from the interval start and up to, but not including the interval end. As an example, for the string <code>tarheel = "carolina"</code>, the interval <code>tarheel[2:5] == "rol"</code> as illustrated below:

<pre>

                                index:      01234567
                               string:     "carolina"
                             interval:        ---O
</pre>

Likewise, the the interval of suffixes begining with 'a' in the following list of sorted circular suffixes shown below is suf[1:3]:

<pre>
                             suf = [ '$carolina',
                                     'a$carolin',
                                     'arolina$c',
                                     'carolina$',
                                     'ina$carol',
                                     'lina$caro',
                                     'na$caroli',
                                     'olina$car',
                                     'rolina$ca'  ]
</pre>

This applies to the genomic intervals that you are asked to use and report in **Problem 4**.

In [3]:
import itertools
import math
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

def loadFasta(filename):
    """ Parses a classically formatted and possibly 
        compressed FASTA file into two lists. One of 
        headers and a second list of sequences.
        The ith index of each list correspond."""
    if (filename.endswith(".gz")):
        fp = gzip.open(filename, 'r')
    else:
        fp = open(filename, 'r')
    # split at headers
    data = fp.read().split('>')
    fp.close()
    # ignore whatever appears before the 1st header
    data.pop(0)     
    headers = []
    sequences = []
    for sequence in data:
        lines = sequence.split('\n')
        headers.append(lines.pop(0))
        # add an extra "+" to make string "1-referenced"
        sequences.append('+' + ''.join(lines))
    return (headers, sequences)

class Graph:
    def __init__(self, vlist=[]):
        """ Initialize a Graph with an optional vertex list """
        self.index = {v:i for i,v in enumerate(vlist)}
        self.vertex = {i:v for i,v in enumerate(vlist)}
        self.edge = []
        self.edgelabel = []
    def addVertex(self, label):
        """ Add a labeled vertex to the graph """
        index = len(self.index)
        self.index[label] = index
        self.vertex[index] = label
    def addEdge(self, vsrc, vdst, label='', repeats=True):
        """ Add a directed edge to the graph, with an optional label. 
        Repeated edges are distinct, unless repeats is set to False. """
        e = (self.index[vsrc], self.index[vdst])
        if (repeats) or (e not in self.edge):
            self.edge.append(e)
            self.edgelabel.append(label)
    def hamiltonianPath(self):
        """ A Brute-force method for finding a Hamiltonian Path. 
        Basically, all possible N! paths are enumerated and checked
        for edges. Since edges can be reused there are no distictions
        made for *which* version of a repeated edge. """
        for path in itertools.permutations(sorted(self.index.values())):
            for i in range(len(path)-1):
                if ((path[i],path[i+1]) not in self.edge):
                    break
            else:
                return [self.vertex[i] for i in path]
        return []
    def SearchTree(self, path, verticesLeft):
        """ A recursive Branch-and-Bound Hamiltonian Path search. 
        Paths are extended one node at a time using only available
        edges from the graph. """
        if (len(verticesLeft) == 0):
            self.PathV2result = [self.vertex[i] for i in path]
            return True
        for v in verticesLeft:
            if (len(path) == 0) or ((path[-1],v) in self.edge):
                if self.SearchTree(path+[v], [r for r in verticesLeft if r != v]):
                    return True
        return False
    def hamiltonianPathV2(self):
        """ A wrapper function for invoking the Branch-and-Bound 
        Hamiltonian Path search. """
        self.PathV2result = []
        self.SearchTree([],sorted(self.index.values()))                
        return self.PathV2result
    def degrees(self):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in self.edge:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return inDegree, outDegree
    def verifyAndGetStart(self):
        inDegree, outDegree = self.degrees()
        start = 0
        end = 0
        for vert in self.vertex.keys():
            ins = inDegree.get(vert,0)
            outs = outDegree.get(vert,0)
            if (ins == outs):
                continue
            elif (ins - outs == 1):
                end = vert
            elif (outs - ins == 1):
                start = vert
            else:
                start, end = -1, -1
                break
        if (start >= 0) and (end >= 0):
            return start
        else:
            return -1
    def eulerianPath(self):
        graph = [(src,dst) for src,dst in self.edge]
        currentVertex = self.verifyAndGetStart()
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                for edge in graph:
                    try:
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print("There is no path!")
                    return False
        return path
    def eulerEdges(self, path):
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeList = []
        for i in range(len(path)-1):
            edgeList.append(self.edgelabel[edgeId[path[i],path[i+1]].pop()])            
        return edgeList
    def render(self, highlightPath=[]):
        """ Outputs a version of the graph that can be rendered
        using graphviz tools (http://www.graphviz.org/)."""
        edgeId = {}
        for i in range(len(self.edge)):
            edgeId[self.edge[i]] = edgeId.get(self.edge[i], []) + [i]
        edgeSet = set()
        for i in range(len(highlightPath)-1):
            src = self.index[highlightPath[i]]
            dst = self.index[highlightPath[i+1]]
            edgeSet.add(edgeId[src,dst].pop())
        result = ''
        result += 'digraph {\n'
        result += '   graph [nodesep=2, size="10,10"];\n'
        for index, label in self.vertex.iteritems():
            result += '    N%d [shape="box", style="rounded", label="%s"];\n' % (index, label)
        for i, e in enumerate(self.edge):
            src, dst = e
            result += '    N%d -> N%d' % (src, dst)
            label = self.edgelabel[i]
            if (len(label) > 0):
                if (i in edgeSet):
                    result += ' [label="%s", penwidth=3.0]' % (label)
                else:
                    result += ' [label="%s"]' % (label)
            elif (i in edgeSet):
                result += ' [penwidth=3.0]'                
            result += ';\n'                
        result += '    overlap=false;\n'
        result += '}\n'
        return result

**Problem #1:** Construct a minimal string of DNA nucleotides includes every 4-base k-mer exactly once *except* "ACAT" which does not appear. How long is your string? How long is the minimal string of DNA nucleotides that includes "ACAT" exactly twice and all other 4-mers exactly once?

In [4]:
# This cell is provided for work on problem 1. It will not be graded.
poss = []
for number in itertools.product(("A", "C", "G", "T"), repeat = 4):
    poss.append(''.join(number))
poss.remove("ACAT")

nodes = sorted(set([code[:-1] for code in poss] + [code[1:] for code in poss]))
G = Graph(nodes)

for code in poss:
   # Here I give each edge a label
   G.addEdge(code[:-1],code[1:],code)

%timeit G.eulerianPath()
path = G.eulerianPath()
print(nodes)
print(path)
edges = G.eulerEdges(path)
print(edges)
pathString = edges[0] + ''.join([edges[i][-1] for i in range(1,len(edges))])
print("pathString length = ", len(pathString))
print(pathString)

2.36 ms ± 887 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
[19, 12, 51, 12, 48, 0, 0, 1, 5, 21, 21, 22, 26, 42, 42, 43, 47, 63, 63, 62, 59, 47, 62, 58, 43, 46, 59, 46, 58, 42, 41, 38, 26, 43, 45, 54, 26, 41, 37, 22, 27, 46, 57, 38, 27, 47, 61, 54, 27, 45, 53, 22, 25, 38, 25, 39, 29, 55, 29, 54, 25, 37, 21, 23, 30, 58, 41, 39, 30, 59, 45, 55, 30, 57, 37, 23, 31, 62, 57, 39, 31, 63, 61, 55, 31, 61, 53, 23, 29, 53, 21, 20, 17, 5, 22, 24, 33, 5, 23, 28, 49, 5, 20, 16, 1, 6, 25, 36, 17, 6, 26, 40, 33, 6, 27, 44, 49, 6, 24, 32, 1, 7, 29, 5

In [5]:
# This cell is provided for work on problem 1. It will not be graded.
poss = []
for number in itertools.product(("A", "C", "G", "T"), repeat = 4):
    poss.append(''.join(number))
poss.append("ACAT")

nodes = sorted(set([code[:-1] for code in poss] + [code[1:] for code in poss]))
G2 = Graph(nodes)

for code in poss:
   # Here I give each edge a label
   G2.addEdge(code[:-1],code[1:],code)

%timeit G2.eulerianPath()
path = G2.eulerianPath()
print("nodes: ", nodes)
print("path: ", path)
edges = G2.eulerEdges(path)
print("edges: ", edges)
pathString = edges[0] + ''.join([edges[i][-1] for i in range(1,len(edges))])
print("pathString length = ", len(pathString))
print(pathString)

2.27 ms ± 106 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
nodes:  ['AAA', 'AAC', 'AAG', 'AAT', 'ACA', 'ACC', 'ACG', 'ACT', 'AGA', 'AGC', 'AGG', 'AGT', 'ATA', 'ATC', 'ATG', 'ATT', 'CAA', 'CAC', 'CAG', 'CAT', 'CCA', 'CCC', 'CCG', 'CCT', 'CGA', 'CGC', 'CGG', 'CGT', 'CTA', 'CTC', 'CTG', 'CTT', 'GAA', 'GAC', 'GAG', 'GAT', 'GCA', 'GCC', 'GCG', 'GCT', 'GGA', 'GGC', 'GGG', 'GGT', 'GTA', 'GTC', 'GTG', 'GTT', 'TAA', 'TAC', 'TAG', 'TAT', 'TCA', 'TCC', 'TCG', 'TCT', 'TGA', 'TGC', 'TGG', 'TGT', 'TTA', 'TTC', 'TTG', 'TTT']
path:  [4, 16, 0, 0, 1, 4, 17, 4, 18, 8, 34, 8, 35, 12, 51, 12, 50, 8, 32, 0, 2, 8, 33, 4, 19, 12, 48, 0, 3, 12, 49, 4, 19, 13, 54, 24, 35, 13, 55, 28, 51, 13, 52, 16, 1, 5, 22, 26, 42, 42, 43, 47, 63, 63, 62, 59, 47, 62, 58, 43, 46, 59, 46, 58, 42, 41, 38, 26, 43, 45, 54, 26, 41, 37, 22, 27, 46, 57, 38, 27, 47, 61, 54, 27, 45, 53, 22, 24, 33, 5, 23, 30, 58, 41, 39, 30, 59, 45, 55, 30, 57, 37, 23, 31, 62, 57, 39, 31, 63, 61, 55, 31, 61, 53, 23, 28, 49, 5, 20, 16, 2, 

**Problem #2:** A file of 31-mers simulating short reads from a genome can be <a href="http://csbio.unc.edu/mcmillan/Comp555S21/reads.fa" download="reads.fa">downloaded here</a>. How many distinct nodes appear in the De Bruijn graph that represents these 31-mers as edges? How many nodes are semi-balenced? How many nodes are balanced? How many are balanced with both in-degrees and out-degrees equal to 1?

In [6]:
# This cell is provided for your own use in answering problem 2. It will not be graded.
hdr, seq = loadFasta("data/reads.fa")

nodes = sorted(set([code[1:-1] for code in seq] + [code[2:] for code in seq]))
G3 = Graph(nodes)

for code in seq:
   # Here I give each edge a label
   G3.addEdge(code[1:-1],code[2:],code)

print("nodes length: ", len(nodes))
inDeg, outDeg = G3.degrees()

nodes length:  19963


In [7]:
#distinct nodes
distinct = {}
for x in nodes:
    if (distinct.get(x) is None):
        distinct[x] = 1

print(len(distinct))

19963


In [8]:
# sort through nodes
semiBal = []
bal = []
balOne = []

print(len(inDeg))
print(len(outDeg))
for i in range(len(nodes)):
    nodeIn = inDeg.get(i)
    nodeOut = outDeg.get(i)
    if (nodeIn is None):
        nodeIn = 0
    if (nodeOut is None):
        nodeOut = 0

    if (nodeIn == nodeOut):
        bal.append(i)
        if (nodeIn == 1):
            balOne.append(i)
    elif (abs(nodeIn - nodeOut) == 1):
        semiBal.append(i)
    
print("semi-balanced: ", len(semiBal))
print("balanced: ", len(bal))
print("balanced with degrees equal 1: ", len(balOne))

19962
19962
semi-balanced:  2
balanced:  19961
balanced with degrees equal 1:  19953


**Problem #3:** What is the length of the Eulerian path that can be constructed in the De Bruijn graph described in Problem #2? It should be an assembled contig from the genome <a href="http://csbio.unc.edu/mcmillan/Comp555S21/data/Chromo.fa" download="Chromo.fa">Chromo.fa</a>. How long did it take to assemble it using Jupyter %time? Find the starting position of this assembled sequence in that genome.

In [9]:
# This cell is provided for your own use in answering problem 3. It will not be graded.
# this uses read.fa, not Chromo
%timeit G3.eulerianPath()
path = G3.eulerianPath()
edges = G3.eulerEdges(path)
pathString = edges[0] + ''.join([edges[i][-1] for i in range(1,len(edges))])

hdr, seq = loadFasta("data/Chromo.fa")

15.6 s ± 556 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
20001


In [11]:
print(len(path))
print(seq[0].index(pathString[1:]))

19971
56800000


**Problem #4:** Construct a suffix array of the substring from <a href="http://csbio.unc.edu/mcmillan/Comp555S21/Chromo.fa" download="Chromo.fa">Chromo.fa</a> from positions 30000000 to 30050000 with a '$' appended. Use it to find all the positions of the substring "ATGCAT" within this substring (include the offset of 30000000), and their interval within the suffix array. How many times does "CAT" appear in the specified substring?

In [32]:
# This cell is provided for your own use in answering problem 4. It will not be graded.
# suffixArray() from lecture 11
def suffixArray(string):
    return sorted(range(len(string)), key=lambda x: string[x:])

def findFirst(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:] < pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

# remember that last returns the position after the last location because it is non-inclusive
def findLast(pattern, text, suffixarray):
    lo, hi = 0, len(text)
    while (lo < hi):
        middle = (lo+hi)//2
        if text[suffixarray[middle]:suffixarray[middle]+len(pattern)] <= pattern:
            lo = middle + 1
        else:
            hi = middle
    return lo

text = seq[0][30000000:30050000] + '$'
sa = suffixArray(text)
first = findFirst("ATGCAT", text, sa)
last = findLast("ATGCAT", text, sa)
print("ATGCAT")
print(first, last)
for suffix in sa[first:last]: 
    print("%3d: %s" % ((suffix+30000000), text[suffix:suffix+8]))
print(last - first, "times")

#"CAT"
first = findFirst("CAT", text, sa)
last = findLast("CAT", text, sa)
print()
print("CAT")
print(first, last)
print(last - first, "times")


ATGCAT
13657 13666
30048108: ATGCATAA
30009927: ATGCATAC
30025162: ATGCATGG
30031223: ATGCATGG
30024494: ATGCATGT
30022117: ATGCATTA
30013403: ATGCATTG
30031774: ATGCATTG
30039059: ATGCATTT
9 times

CAT
18221 19243
1022 times


**Problem #5:** Use your suffix array from Problem #4 to construct a BWT of the given 50,000 bp fragment. Once the BWT is found encode any run of more than 1 base as an integer followed by the base. What is the total length of this compressed BWT? What is the average run-length of 'A', 'C', 'G' and 'T' nucleotides?  

In [36]:
# This cell is provided for your own use in answering problem 5. It will not be graded.
def BWTfromSuffixArray(text, suffixarray):
    return ''.join(text[i-1] for i in suffixarray)

def compress(text):
    c = ""
    i = 0
    while i < len(text):
        if ((i+1) >= len(text)):
            c = c + text[i]
            i += 1
        elif (text[i+1] != text[i]):
            c = c + text[i]
            i += 1
        else:
            count = 1
            while (text[i+count] == text[i]):
                count+=1
            c = c + "%s" % count + text[i]
            i = i + count
    return c

def runLength(text, char):
    num = ''
    total = 0
    count = 0
    x = 0
    while x < len(text):
        while text[x].isdigit():
            num += text[x]
            x += 1
    
        if(text[x] != char):
            num = ''
        elif text[x] == char:
            if(num == ''):
                total += 1
            else:
                numInt = int(num)
                total += numInt
            count += 1
        x += 1 
    # print(total)
    # print(count)
    return (total/count)     

# testCompress = compress("amnnn$lcpmnapaaaaaaala")
# print(testCompress)
# print(runLength(testCompress, 'a'))

bwt = BWTfromSuffixArray(text, sa)
compressBwt = compress(bwt)
print(len(compressBwt))
print("A: ", runLength(compressBwt, 'A'))
print("C: ", runLength(compressBwt, 'C'))
print("G: ", runLength(compressBwt, 'G'))
print("T: ", runLength(compressBwt, 'T'))


45510
A:  1.4971880659613002
C:  1.2900836647922096
G:  1.2484743963916158
T:  1.4870760065340636


---

## Instructions for submitting your problem set

When you are ready to submit a version of your problem set, follow the instructions below.

1. Press [Save and Checkpoint] on the *File* menu of your Jupyter notebook.
2. Press the link below, which will take you to a website for submitting your problem set.
3. Choose the ***correct problem set number*** from the pull-down, else you might overwrite an earlier submission.
4. Enter in your onyen and PID in the form provided, then upload your submission.

Click [here to submit](http://csbio.unc.edu/mcmillan/index.py?run=PS.upload) your completed problem set

**Instructions for resubmissions:**

1. You may resubmit as many times as you like before the deadline. 
2. Resubmissions *always* overwrite any earlier submissions. 
3. If you resubmit after the due date, you will be warned of any penalties. 
4. Problem sets will not be regraded.